## **本ノートブックの内容**

- 感染者データの取得・テーブル作成
- ヒストグラムの作成
- groupbyによるデータのグループ化
- 相関係数の算出


In [ ]:
# モジュールのimport
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.dates as mdate

pd.set_option('display.max_columns', None) # pandasオプション：列データを全て表示

# データフレームdfにOur World in Dataのデータを読み込む
df = pd.read_json("https://covid.ourworldindata.org/data/owid-covid-data.json")
df = df[df.columns.drop(list(df.filter(regex='OWID')))] # OWIDデータを除外

df

In [ ]:
# 難：日付データ"data"から新たなデータフレームを作成

countries = df.columns # 列の国名コード
date_keys = pd.DataFrame(df['USA']['data']).columns # dataに含まれるcolumn名（アメリカより取得）

df2 = pd.DataFrame(columns = countries, index = date_keys) # index：感染情報，column：国名コードとなるdf2を作成する

for column in countries:
    for indexs in date_keys:
        try:
            df2[column][indexs] = df[column]['data'][-7][indexs] # 1週間前のデータがあれば格納
        except LookupError:
            df2[column][indexs] = 'NaN' # なければNaN（not-a-number）を格納

# 型変換：objectをfloatへと変換
dft  = df.T.apply(pd.to_numeric, errors='ignore')
dft2 = df2.T.apply(pd.to_numeric, errors='coerce')
            
# 二つのデータフレームをマージ
df_all = pd.concat([dft,dft2], axis=1)

df_all # df_allの表示

In [ ]:
df_all.info() # テーブルの概要を確認

# 各columnの詳細情報は以下を確認すること
# URL: https://github.com/owid/covid-19-data/tree/master/public/data

In [ ]:
df_all.describe() # describeと付けると統計情報を確認することができる

In [ ]:
# 具体的に特定の値を算出したいときは以下のようにアクセスできる
print(df_all['population'].median(), df_all['population'].mean()) # 中央値，平均値
print(df_all['population'].min(), df_all['population'].max()) # 最小値，最大値

In [ ]:
# 最小値，最大値を取る行のindexを取得
print(df_all['population'].idxmax())
print(df_all['population'].idxmin())

In [ ]:
# histを用いるとヒストグラムを作成できる
# bins: 区間（ビン）の数
# density: 正規化あり
df_all['median_age'].hist(bins=15, density=True)

In [ ]:
# 大陸ごとのデータを抽出
df_asia = df_all[df_all['continent'] == 'Asia']
df_oceania = df_all[df_all['continent'] == 'Oceania']

# ヒストグラムを重ね合わせるには複数回 hist を書く
# range: ヒストグラムを作成する範囲 [最小値，最大値]
# alpha: ヒストグラムの透明度．重ね合わせるため透明度を上げて裏も見せるために必要
ax = df_asia['median_age'].hist(bins=12, density=True, range=[10, 50], alpha=0.5, label='Asia')
df_oceania['median_age'].hist(bins=12, density=True, range=[10, 50], alpha=0.5, label='Oceania')

ax.legend() # 凡例の追加

In [ ]:
# ヒストグラムを大陸毎に別々にグラフを作りたい場合
# byを用いるとグループ毎にヒストグラムを作成
# figsize: 図のサイズ指定

df_all.hist(column='median_age', by=df_all['continent'], bins=10, range=[10, 50], density=True, figsize=(8, 8))

In [ ]:
# 2つのcolumnのデータに対する相関値の算出
df_all[['population_density', 'total_cases_per_million']].corr()

In [ ]:
# groupbyを用いると指定した列の情報でテーブルを分類できる
# ここでは大陸（'continent'）を用いて情報を分類
grouped = df_all.groupby('continent')

# describeを用いて統計情報の閲覧
grouped.describe()

In [ ]:
# 特定の列について，グループ間で比較したい場合
print(grouped['gdp_per_capita'].mean())

# 特定の列について，図を用いて比較したい場合
grouped['gdp_per_capita'].mean().plot.bar()

In [ ]:
# annotateを使うと国名コードを表記できる

# keyを2つ選択する
key1 = 'population'
key2 = 'total_deaths'

# key1，key2について散布図を作成
ax = df_all.plot.scatter(x=key1, y=key2, logx=True, logy=True)

# df_allについてfor文
for index, row in df_all.iterrows():
    ax.annotate(index, (row[key1], row[key2])) # 座標(row[key1], row[key2])の位置に国名コードであるindexを表示
    
    # 詳しく知りたい人へ（読み飛ばしても可）
    # index: 行のindexを格納．この場合は国名コードであるアルファベット3文字
    # row: ある行の全ての情報が格納．例えばrow['population']とすると，その行の人口を取得．
    # 各行へのアクセスをfor文により実現している

In [ ]:
# 難：大陸毎に色をつけた散布図

# 図のテンプレートを準備
fig, ax = plt.subplots()
ax.set_xscale('log')
ax.set_yscale('log')

# 横軸・縦軸の項目指定
key1 = 'gdp_per_capita'
key2 = 'aged_70_older'

# 色指定：Matpolotlibの標準色を採用; 辞書型
colors = {'Africa':'#1f77b4', 'Asia':'#ff7f0e', 'Europe':'#2ca02c', 'North America':'#d62728', 'Oceania':'#9467bd', 'South America':'#8c564b'}

# for文により大陸毎の散布図をループ
for key, group in grouped:
    group.plot.scatter(x=key1, y=key2, ax=ax, label=key, color=colors[key])
    
    # 詳しく知りたい人へ（読み飛ばしても可）
    # grouped: groupbyを用いて作成したグループ後のデータ．for文に用いる
    # key: groupbyで用いた分類名．ここでは大陸名
    # group: groupbyで分類された後のデータ．ここでは大陸毎のデータ
    # for文により各大陸のデータをループする

#### **Q3-1**． 100万人当たりの感染者数（total_cases_per_million）が最も多い国名と，その値を表示せよ

In [ ]:
# A3-1


#### **Q3-2**． 日本の100万人当たりの感染者数（total_cases_per_million）を表示せよ．何人に一人感染しているか？

In [ ]:
# A3-2


#### **Q3-3**． テーブルを総感染者数（total_cases）でソートし，感染者数トップ5を確認せよ
ヒント：降順にソートする場合は `df.sort_values('column名', ascending=False)`

In [ ]:
# A3-3


#### **Q3-4**． 100万人当たりの感染者数（total_cases_per_million）をヒストグラムにして表示せよ

In [ ]:
# A3-4


#### **Q3-5**． 横軸に70歳以上の割合（aged_70_older），縦軸に100万人当たりの死者数（total_deaths_per_million）を取り，両対数グラフで表示せよ．人口構成に比して死亡率が高めの国はどこか．

In [ ]:
# A3-5


#### **Q3-6**． 前問3-5の2項目について，相関係数を算出せよ

In [ ]:
# A3-6


#### **Q3-7**． 100万人当たりの総感染者数について，アジア・北米・ヨーロッパの国々のヒストグラムを重ね合わせて比較せよ

In [ ]:
# A3-7


#### **Q3-8**． 100万人当たりの総感染者数について，最も多い・少ない大陸はどこか．数字およびグラフ表示により比較せよ

In [ ]:
# A3-8


#### **Q3-9**． 横軸に総人口（population），縦軸に100万人当たりの総感染者数（total_cases_per_million）を取り，両対数グラフで表示せよ．大陸毎に違う色で可視化したときに，大陸毎に見られる傾向を分析せよ

In [ ]:
# A3-9


#### **Q3-10**． 難：WHO・国連の定義より，65歳以上の人口（老年人口）の割合が7%を超えると「高齢化社会（aging society）」，14%を超えると「高齢社会（aged society）」，21%を超えると「超高齢化社会（super aging society）」と呼ぶ．以下に答えよ．
A. 難：4つのカテゴリ（超高齢化社会・高齢社会・高齢化社会・その他）に分類される国の数を答えよ  
ヒント1：論理和が必要となる  
ヒント2：groupbyを使った方法もできますが，難しいので今回は使わない方法を採用すること  
B. 超高齢化社会に属する国を列挙せよ  
C. 4つのカテゴリにおける100万人当たりの総感染者数の平均値を算出し比較せよ  
D. 難：4つのカテゴリを色分けして上で，横軸に一人当たりの国内総生産，縦軸に100万人当たりの総感染者数を示せ
ヒント：上のテクニックを組み合わせると可能  

In [ ]:
# A3-10
